# training.ipynb

Training implementation.

Author: Connacher Murphy

In [18]:
# Libraries
import pest_classification as pest

# import os
# from sklearn.metrics import accuracy_score
import timm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from types import SimpleNamespace

In [2]:
# Split df into test and training
df = pest.df
test_df = df[df["set"] == "test_set"]
train_df = df[df["set"] == "train_set"]

In [3]:
# Dataloader
config = SimpleNamespace(**{})

config.batch_size = 32

config.image_dir = os.path.expanduser("~/data/ccmt/CCMT Dataset-Augmented")
config.image_size = 256

train_dataset = pest.AugmentedCCMT(config, train_df)
test_dataset = pest.AugmentedCCMT(config, test_df)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=0,
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=0,
)


In [4]:
# config.num_classes = len(pest.crop_classes["Maize"])
config.num_classes = 2
config.backbone = "resnet18"

model = timm.create_model(config.backbone, pretrained=True, num_classes=config.num_classes)

In [5]:
X = torch.randn(config.batch_size, 3, config.image_size, config.image_size)
y = model(X)
print(y.shape)
print(y)

torch.Size([32, 2])
tensor([[ 0.0837,  0.1922],
        [ 0.1555,  0.2691],
        [ 0.2127,  0.2239],
        [ 0.1083,  0.1553],
        [ 0.0760,  0.1259],
        [ 0.0967,  0.1038],
        [ 0.0200,  0.2212],
        [ 0.1437,  0.2937],
        [-0.0025,  0.0244],
        [ 0.1894,  0.1958],
        [ 0.0890,  0.1952],
        [ 0.0943,  0.2816],
        [ 0.0724,  0.2766],
        [ 0.0176,  0.0676],
        [ 0.1559,  0.1034],
        [ 0.1492,  0.1789],
        [ 0.2803,  0.2870],
        [ 0.1225,  0.1178],
        [ 0.1088,  0.1622],
        [ 0.1486,  0.2833],
        [ 0.0809,  0.2048],
        [-0.0051,  0.1223],
        [ 0.1950,  0.1478],
        [-0.0695,  0.2053],
        [ 0.1435,  0.1495],
        [ 0.0272,  0.2351],
        [ 0.0989,  0.1290],
        [ 0.1057,  0.0906],
        [ 0.2808,  0.1124],
        [ 0.1429,  0.0881],
        [-0.0670,  0.0997],
        [ 0.1567,  0.1621]], grad_fn=<AddmmBackward0>)


In [6]:
config.criterion = nn.CrossEntropyLoss()

In [7]:
config.lr = 1e-4

optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=0.0)

In [9]:
torch.cuda.is_available()

False

In [10]:
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
model.to(config.device)

def train(dataloader, model, optimizer, config):
    for epoch in range(config.num_epochs):
        print(f"Epoch {epoch + 1}")
        epoch_loss = pest.train_epoch(dataloader, model, optimizer, config)
        print(epoch_loss)

torch.Size([32, 3, 256, 256])
torch.Size([32])


0.7097724676132202

In [14]:
config.num_epochs = 3

In [15]:
train(train_dataloader, model, optimizer, config)

Epoch 1
torch.Size([32, 3, 256, 256])
torch.Size([32])
0.7061287760734558
Epoch 2
torch.Size([32, 3, 256, 256])
torch.Size([32])
0.6771159768104553
Epoch 3
torch.Size([32, 3, 256, 256])
torch.Size([32])
0.6556016206741333
